In [ ]:
!pip install transformers==4.49 diffusers==0.32.2

In [ ]:
!wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/utils.py
!wget https://raw.githubusercontent.com/thiagohersan/media-landscape/refs/heads/main/python/imgs/landscape_00.jpg

In [ ]:
import os
os.environ["GEMINI_API_KEY"] = ""

import torch

from PIL import Image as PImage

from utils import get_img_description
from utils import get_pipeline, get_input_images

In [ ]:
# "runwayml/stable-diffusion-inpainting",
# "stable-diffusion-v1-5/stable-diffusion-inpainting",
# "stabilityai/stable-diffusion-2-inpainting",

pipe = get_pipeline("stable-diffusion-v1-5/stable-diffusion-inpainting")


In [ ]:
img00 = PImage.open("./landscape_00.jpg")
img00 = PImage.open("./manhattan.jpg")
img, mask = get_input_images(img00, keep_width=256, size=(1440, 512))

# display(img)
# display(mask)

In [ ]:
prompt = get_img_description(img00)
anthropoprompt = f"post apocalyptic version of {prompt}, but with things on fire, floods and trash everywhere"
anthropoprompt

In [ ]:
generator = torch.Generator(device="cuda").manual_seed(1010)

output = pipe(
  prompt=anthropoprompt,
  negative_prompt="repetitive, distortion, glitch, borders, stretched, frames, breaks, multiple rows",
  image=img,
  mask_image=mask,
  width=img.size[0], height=img.size[1],
  guidance_scale=16.0,
  num_inference_steps=24,
  generator=generator
)

display(output.images[0])
